In [3]:
from bs4 import BeautifulSoup as bs
import requests
import time
import pandas as pd
import random

In [2]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/92.0.902.78 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/91.0.4472.164 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/91.0"
]

headers = {'User-Agent': random.choice(user_agents)}

# Creating Filters and Features Dict

Os and products(tablet, watch etc.) that we do not want to scrape are in the discarded lists.

In scraping process we append scraped features in feature dict.

In [ ]:
main_URL = ''

discarded_os = ['Android Wear','watchOS','Proprietary','HarmonyOS','ColorOS','Feature phone','KaiOS','Microsoft Windows']
discarded_phone_name = ['Watch','Tab','Pad','IdeaTab','Tablet','iPad','View','Gear', 'FHD REL']

data = {'phone_name':[], 'brand':[], 'os':[], 'inches':[], 'resolution':[], 'battery':[], 'battery_type':[], 'ram':[], 'announcement_date':[], 
        'weight':[], 'video':[],'internal':[], 'price':[]}

page_links = []
phone_links = []

# Scraping Main Page

In [ ]:
brand_links = []
brands= []

r = requests.get(main_URL, headers = {'User-Agent': random.choice(user_agents)})
main_soup = bs(r.text, 'html.parser')

div_table = main_soup.find('div',class_='brandmenu-v2')
brands_ul = div_table.find('ul')

for li in brands_ul.find_all('li'):
    brand_links.append(li.find('a',href=True)['href'])
    brands.append(li.find('a').text)

# Discarding unpopular brands
del brand_links[16:]
del brand_links[-2]
del brand_links[-2]
del brand_links[6]

del brands[16:]
del brands[-2]
del brands[-2]
del brands[6]

# Scraping Process

In [ ]:
from numpy import nan

r = requests.get('')
brand_soup = bs(r.text, 'html.parser')
page_div = brand_soup.find('div', class_='pages')

# Appending every page links in brand link
for a in page_div.find_all('a',href=True):
    page_links.append(a['href'])

    # Inserting first page link to page_links list
first_link = list(page_links[0])
first_link[-5] = '1'
first_link = "".join(first_link)
page_links.insert(0, first_link)
page_links = page_links[:2]

print(page_links)
# Scraping every phone link in page link
for page_link in page_links:
    time.sleep(1)
    r3 = requests.get(main_URL+'/'+page_link, headers = {'User-Agent': random.choice(user_agents)})
    page_soup = bs(r3.text,'html.parser')
    phone_div = page_soup.find('div',class_='makers')
    phone_ul = phone_div.find('ul')

    # Appending every phone link to phone_links lists in page link
    for li in phone_ul.find_all('li'):
        phone_links.append(li.find('a',href=True)['href'])

    # Scraping phone feature in every phone links
    for phone_link in phone_links:
        time.sleep(1)
        r2 = requests.get(main_URL+'/'+phone_link,headers = {'User-Agent': random.choice(user_agents)})
        phone_soup = bs(r2.text, 'html.parser')

        if phone_soup.find('div',id='specs-list').find_all('table')[1].find_all('tr')[1].find('td',class_='nfo').text.split('.')[0] == 'Available':
            if ((any(substring in phone_soup.find('h1',class_='phone-name-title').text for substring in discarded_phone_name)) | 
                (any(substring in phone_soup.find_all('span', class_='spe_accent')[2].text.split(',')[0] for substring in discarded_os))):
                continue
            else:
                data['phone_name'].append(phone_soup.find('h1',class_='phone-name-title').text)
                data['os'].append(phone_soup.find_all('span', class_='spe-accent')[2].text.split(',')[0])
            
            data['brand'].append('Sony')

            if phone_soup.find('li', class_='help-display').find('strong',class_='accent').find('span') == None:
                data['inches'].append(nan)
            else:
                data['inches'].append(phone_soup.find('li', class_='help-display').find('strong',class_='accent').find('span').text)
            
            if phone_soup.find('li', class_='help-display').find('div') == None:
                data['resolution'].append(nan)
            else:
                data['resolution'].append(phone_soup.find('li', class_='help-display').find('div').text)

            if phone_soup.find('strong', class_='ac_battery').find('span') == None:
                data['battery'].append(nan)
            else:
                data['battery'].append(phone_soup.find('strong', class_='ac_battery').find('span').text)

            if phone_soup.find('li',class_='battery').find('div') == None:
                data['battery_type'].append(nan)
            else:
                data['battery_type'].append(phone_soup.find('li',class_='help-battery').find('div').text)

            temp_ram =''
            if phone_soup.find('strong', class_='accent-expansion').find_all('span') == None:
                data['ram'].append(nan)
            else:
                for text in phone_soup.find('strong', class_='accent accent-expansion').find_all('span'):
                    temp_ram += text.get_text()
                data['ram'].append(temp_ram)
            
            if phone_soup.find('div', id='spe-list').find_all('table')[1].find('td', class_='nfo') == None:
                data['announcement_date'].append(nan)
            else:
                data['announcement_date'].append(phone_soup.find('div', id='specs-list').find_all('table')[1].find('td', class_='nfo').text)
            
            if phone_soup.find('div', id='specs-list').find_all('table')[2].find_all('tr')[1].find('td', class_='nfo') == None:
                data['weight'].append(nan)
            else:
                data['weight'].append(phone_soup.find('div', id='specs-list').find_all('table')[2].find_all('tr')[1].find('td', class_='nfo').text)
            
            all_video_tr = phone_soup.find('div', id='specs-list').find_all('table')[6].find_all('tr')
            for video_tr in all_video_tr:
                if video_tr.find('td', class_='ttl').text == 'Video':
                    data['video'].append(video_tr.find('td', class_='nfo').text)
                        
            if phone_soup.find('div', id='sp_list').find_all('table')[5].find_all('tr')[1].find('td', class_='nfo') == None:
                data['internal'].append(nan)
            else:
                    data['internal'].append(phone_soup.find('div', id='sp-list').find_all('table')[5].find_all('tr')[1].find('td', class_='nfo').text)

            all_tr = phone_soup.find('div',id='sp-list').find_all('table')[12].find_all('tr')
            for tr in all_tr:
                if tr.find('td',class_='ttl').text == 'Price':
                    if tr.find('td',class_='nfo').text == '':
                        data['price'].append(nan)
                    else:
                        data['price'].append(tr.find('td', class_='nfo').text)
                
                    
        else:
            continue
    # Clearing phone_links list after finished page
    phone_links.clear()          
# Clearing page_links list after finished brand
page_links.clear()          

# Create Data Frame

In [ ]:
df = pd.DataFrame(data)
df.to_csv('../data/....csv', index=False)

In [5]:
import glob
 
# list all csv files only
csv_files = glob.glob('../data/*.{}'.format('csv'))


['../data/google.csv',
 '../data/oppo.csv',
 '../data/honor.csv',
 '../data/lenovo.csv',
 '../data/sony.csv',
 '../data/apple.csv',
 '../data/huawei.csv',
 '../data/oneplus.csv',
 '../data/xiaomi.csv',
 '../data/samsung.csv',
 '../data/vivo.csv',
 '../data/realme.csv',
 '../data/lg.csv']

In [8]:
all_phone = pd.concat([pd.read_csv(file) for file in csv_files ], ignore_index=True)
all_phone['brand'].unique()

array(['Google', 'Oppo', 'Honor', 'Lenovo', 'Sony', 'Apple', 'Huawei',
       'OnePlus', 'Xiaomi', 'Samsung', 'Vivo', 'Realme', 'LG'],
      dtype=object)

In [ ]:
all_phone.to_csv('../data/all_phones.csv', index=False)